In [5]:
import re
import time
import pandas as pd
import numpy as np
import nltk
#from nltk import *
import json

import random
random.seed(10)

from joblib import dump, load

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score


#import tensorflow as tf
from tensorflow import keras

# Ejemplo con Stack de RNNs
from keras.layers import Embedding, SimpleRNN
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, LSTM, SpatialDropout1D
from keras.models import Sequential, load_model
from keras.losses import SparseCategoricalCrossentropy

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score #son metricas - area debajo de la curva roc

from sklearn.model_selection import train_test_split


In [2]:
from tensorflow import keras

# Ejemplo con Stack de RNNs
from keras.layers import Embedding, SimpleRNN
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, LSTM, SpatialDropout1D
from keras.models import Sequential, load_model
from keras.losses import SparseCategoricalCrossentropy

In [2]:
!pip list

Package                      Version
---------------------------- --------
absl-py                      2.1.0
asttokens                    2.4.1
astunparse                   1.6.3
cachetools                   5.3.2
certifi                      2024.2.2
charset-normalizer           3.3.2
colorama                     0.4.6
comm                         0.2.1
debugpy                      1.8.1
decorator                    5.1.1
executing                    2.0.1
flatbuffers                  23.5.26
gast                         0.4.0
google-auth                  2.28.1
google-auth-oauthlib         1.0.0
google-pasta                 0.2.0
grpcio                       1.62.0
h5py                         3.10.0
idna                         3.6
ipykernel                    6.29.2
ipython                      8.22.1
jax                          0.4.24
jedi                         0.19.1
jupyter_client               8.6.0
jupyter_core                 5.7.1
keras                        2.12.0
libc

### Lectura del archivo de tweets filtrados y trasformados.

In [ ]:
corpus_dev_esA = pd.read_csv("tweets_search_etiquetas_clean_test.csv", sep = ";", encoding = "latin-1")
corpus_dev_esA.tail()

### Obtener los vectores de entrada X y de salida Y

In [ ]:
train_idA = corpus_train_esA[corpus_train_esA.columns[0]]
X_train_textA = corpus_train_esA[corpus_train_esA.columns[3]].fillna(' ')
y_train_hsA = corpus_train_esA[[corpus_train_esA.columns[5], corpus_train_esA.columns[6], corpus_train_esA.columns[7]]]
#y_train_hsA = corpus_train_esA[corpus_train_esA.columns[8]]
y_train_hsA = y_train_hsA.values

test_idA = corpus_dev_esA[corpus_train_esA.columns[0]]
X_test_textA = corpus_dev_esA[corpus_dev_esA.columns[3]].fillna(' ')
y_test_hsA = corpus_dev_esA[[corpus_dev_esA.columns[5], corpus_dev_esA.columns[6], corpus_dev_esA.columns[7]]]
#y_test_hsA = corpus_dev_esA[corpus_dev_esA.columns[8]]
y_test_hsA = y_test_hsA.values

### Obtención de los vectores numericos a partir del texto

In [ ]:
cvectorizer = CountVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters 
    ngram_range=(3,5),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df=3,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA)

X_train_cvectorized = cvectorizer.transform(X_train_textA).toarray()
print(X_train_cvectorized.shape)

X_test_cvectorized = cvectorizer.transform(X_test_textA).toarray()
print(X_test_cvectorized.shape)

dump(cvectorizer, 'count_vectorizer.joblib')

In [ ]:
cvectorizer.vocabulary_

In [ ]:
with open("cvectorizer_vocabulary_.json", mode = "w", encoding = "latin-1", ) as f:
    f.write(json.dumps(cvectorizer.vocabulary_, indent = 4, ensure_ascii = False))

### Hiper parametros del modelo

In [ ]:
max_features =  10000 #len(cvectorizer.vocabulary_) or X_train_cvectorized.shape[1] #10000  # tamaño del diccionario de palabras comunes
                      # (número de palabras a utilizar)
maxlen = X_test_cvectorized.shape[1] #1775         # longitud máxima de cada secuencia 
batch_size = 32


### Re neuronal con embedding y capas simples

In [ ]:

model = Sequential()
# Capa embedding
# input_dim : tamaño del vocabulario
# output_dim: dimensión del vector al que se mapea
model.add(Embedding(input_dim = max_features,  output_dim=32, input_length = maxlen)) # output_dim = 32
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))

model.add(Dense(3, activation='softmax'))

model.summary()

#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.compile(optimizer='adam', loss = "categorical_crossentropy", metrics=['accuracy'])


tic = time.time()
history_stackRNN = model.fit(
    X_train_cvectorized, y_train_hsA,
    epochs=15,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=2
)
print('Tiempo de entrenamiento:', time.time()-tic)

# evaluate the model
scores = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))

loss, accuracy = model.evaluate(X_train_cvectorized, y_train_hsA, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

# make predictions
testPredict_stackRNN = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))

### Red neuronal con embedding y redes densas con activación

In [ ]:
embedding_dim = batch_size # 50 #batch_size
#maxlen = 15

model = Sequential()
model.add(Embedding(input_dim = max_features, 
                           output_dim = embedding_dim, 
                           input_length = maxlen))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

history = model.fit(X_train_cvectorized, y_train_hsA,
                    epochs=15,
                    verbose=2,
                    validation_data=(X_test_cvectorized, y_test_hsA),
                    batch_size=batch_size)
                    
loss, accuracy = model.evaluate(X_train_cvectorized, y_train_hsA, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

# make predictions
testPredict_stackRNN = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))



### red neuronal con embedding y long short term memory

In [ ]:

lstm_out = 98

model = Sequential()
model.add(Embedding(input_dim = max_features, output_dim = embedding_dim, input_length = maxlen))
#model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

batch_size = 32
model.fit(X_train_cvectorized, y_train_hsA, epochs = 10, batch_size=batch_size, verbose = 2)

loss, accuracy = model.evaluate(X_train_cvectorized, y_train_hsA, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

# make predictions
testPredict_stackRNN = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))

### Guardar el modelo

In [ ]:
model.save('best_keras_network_model.h5')

In [ ]:
from collections import Counter

input_list =  list(map(lambda x: str(x), y_train_hsA.round()))
input_list =  list(map(lambda x: str(x), y_test_hsA.round()))
input_list =  list(map(lambda x: str(x), testPredict_stackRNN.round()))
c = Counter( input_list )

print( c.items() )

In [ ]:
corpus_dev_esA["predict_test"] = list(map(lambda x: str(x), testPredict_stackRNN.round()))

corpus_dev_esA.to_csv("tweets_search_etiquetas_clean_test_predict.csv", sep="\t", encoding = "latin-1")

In [ ]:

from api_twitter import StreammerTwitter

streammer = StreammerTwitter()
streammer.simulate_search_30("tweets_search_2.txt", True, model_file = "best_keras_network_model.h5", cvectorizer_file = "count_vectorizer.joblib")

#transformed_tweet
